Mid-level
Pasinaudodami Vyriausiosios rinkimų komisijos valdomu API gaukite 2023 m. savivaldybių tarybų ir
merų rinkimų II turo duomenis, kuriais naudodamiesi:
1. raskite kandidatus, kurių gauto palaikymo dalis, skirta balsais paštu, buvo statistiškai
reikšmingai didesnė, lyginant su jų konkurentais toje pačioje rinkimų apygardoje tame
pačiame rinkimų ture;

2. žemėlapyje pavaizduokite rinkimų apylinkes ir partijas, kurių kandidatų tose apylinkėse gauto
palaikymo dalis, skirta balsais paštu, buvo statistiškai reikšmingai didesnė, lyginant su jų
konkurentais toje pačioje rinkimų apylinkėje tame pačiame rinkimų ture.